In [1]:
import os
import gc
import numpy as np
from tqdm import tqdm_notebook, tqdm
import pandas as pd

In [7]:
prob_dir = 'Z:/kaggle/protein/probs/'
DIR = './input/'

submit = pd.read_csv(DIR + '/sample_submission.csv')
def load_npy(dirname):
    prob_npys = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)
        ext = os.path.splitext(full_filename)[-1]
        if ext == '.npy' :#and filename.find('densenet121')>0:# and filename.find(select_prefix2)<0: 

            prob = np.load(full_filename)
            print(filename,'local_score:',prob.item().get('local_score'))
            prob_npys.append(np.load(full_filename))
    return prob_npys



def make_sub(PP, name , SOFT = None):
    prediction = []
    cnt = 0
    if SOFT is None:
        PP_MAX = np.argmax(PP, axis=1)
    else:
        PP_MAX = np.argmax(SOFT[:,:28], axis=1)
    for row in tqdm_notebook(range(submit.shape[0])):    
        str_label = ''
        for col in range(PP.shape[1]):
            if(PP[row, col] < 0):
                str_label += ''
            else:
                str_label += str(col) + ' '
        if str_label == '':
            str_label += str(PP_MAX[row]) + ' '
            cnt+=1
        prediction.append(str_label.strip())
    submit['Predicted'] = np.array(prediction)
    sub_file_path = './subs/' + name+'.csv'
    submit.to_csv(sub_file_path, index=False)
    print('softlabelcnt',cnt)
    return np.array(prediction)

prob_npys = load_npy(prob_dir)

Densenet121(512, 512, 4)_444_8_True_0.002_True_70_15_usemean_only512_multi_v23_addext_50p_dual_wsample_71437_11epoch_f1_6042_8.npy local_score: 0.8110429317236038
Densenet121(512, 512, 4)_777_8_True_0.002_True_70_15_usemean_only512_multi_v23_addext_50p_dual_wsample_71437_0epoch_f1_5459_8.npy local_score: 0.7544803447673966
desne121_777.npy local_score: 0.757304545932076
InceptionResnetV2(512, 512, 4)_189_6_True_0.002_True_70_15_usemean_only512_multi_v23_addext_50p_dual_wsample_71437_12epoch_f1_3663_8.npy local_score: 0.6617097676496021
SeResnet18(512, 512, 4)_123_48_True_0.002_True_30_4_usemean_only512_multi_v23_addext_50p_dual_wsample_71437_4epoch_f1_5391_8.npy local_score: 0.7304241432885006
SeResnet18(512, 512, 4)_777_48_True_0.002_True_30_4_usemean_only512_multi_v23_addext_50p_dual_wsample_71437_13epoch_f1_5006_8.npy local_score: 0.7182992288808233
Xception(512, 512, 4)_555_8_True_0.002_True_30_7_usemean_only512_multi_v23_addext_50p_dual_wsample_71437_14epoch_f1_4260_8.npy local_sc

In [12]:
weights = np.array([1.8
                    ,0.4
                    ,1.4
                    ,0.3
                    ,0.1
                    ,0.1
                   ,0.2
                   ,0.1])

weights = weights/sum(weights)
print(weights)
merge_sum = np.zeros(prob_npys[0].item().get('merge').shape)
soft_sum = np.zeros(prob_npys[0].item().get('soft').shape)

for idx, prob in enumerate(prob_npys):
    print(idx)
    merge_prob = prob.item().get('merge').copy()
    soft_prob =  prob.item().get('soft').copy()
    thr = prob.item().get('best_thr')
    merge_prob  -= thr

    merge_sum += merge_prob*weights[idx]
    soft_sum += soft_prob
    #print(prob.item().get('merge').mean(axis=0))
    
make_sub(merge_sum, 'ensemble_test',soft_sum)

[0.40909091 0.09090909 0.31818182 0.06818182 0.02272727 0.02272727
 0.04545455 0.02272727]
0
1
2
3
4
5
6
7



softlabelcnt 471


array(['2', '5', '0 5 25', ..., '7', '1', '0 25'], dtype='<U13')

In [118]:
merge_sum.mean()

-0.4127361863665077

In [117]:
soft_sum.max()

6.997986137866974